# ChatUnify

This notebook covers how to get started with Unify chat models. When using models through the Unify API endpoint, you get:
- Single sign-on with all providers.
- Dynamic routing to the best provider for your use case.

## Installation

In [ ]:
# install package
!pip install -U langchain-unify langchain-core

## Environment Setup

Make sure to set the following environment variables:

- `UNIFY_API_KEY`: You can get a key in the [Unify Console](https://console.unify.ai/login).

## Usage 
### Chaining Inputs

In [3]:
from langchain_unify.chat_models import ChatUnify
from langchain_core.prompts import ChatPromptTemplate
import os
os.environ["UNIFY_API_KEY"] = "<YOUR API KEY>"
chat = ChatUnify()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates English to French."),
        ("human", "Translate this sentence from English to French. {english_text}."),
    ]
)
chain = prompt | chat
chain.invoke({"english_text": "Hello, how are you?"})

AIMessage(content="  Sure! Here's the translation:\n\nBonjour, comment vas-tu ?")

### Streaming and SSO
One of the advantages of using Unify is that through a single api key, you can access endpoints from different providers like Perplexity, Together AI, OctoAI and more.

In [6]:
chat_together = ChatUnify(model="llama-2-70b-chat@together-ai")
print("Together stream: ")
for chunk in chat_together.stream("Hello Together, where are you from?"):
    print(chunk.content, end="")

chat_anyscale = ChatUnify(model="llama-2-70b-chat@anyscale")
print("\nAnyscale stream: ")
for chunk in chat_anyscale.stream("What's 9 + 10?"):
    print(chunk.content, end="")

Together stream: 
  Hello! I'm an AI, I don't have a physical location or a specific country of origin. I was created by a group of researcher at Meta AI and my primary function is to assist and converse with users like you, answering questions and providing information on a wide range of topics. I'm here to help and provide assistance, so feel free to ask me anything!!
Anyscale stream: 
  Sure! 9 + 10 is 19.

### Batching and Dynamic Routing
You may also want to use `mixtral-8x7b-instruct-v0.1` with the fastest output speed, and are not concerned with which provider you are using. Unify's dynamic routing gets this data from [our live benchmarks](https://unify.ai/hub/mixtral-8x7b-instruct-v0.1) and routes you to the provider that is performing better than the rest at that instant. 

You can see how to configure dynamic batching [here](https://unify.ai/docs/hub/concepts/runtime_routing.html#how-to-use-it).

As expected, you can also batch your queries while using dynamic routing!

In [11]:
from langchain_core.messages import HumanMessage
messages = ["Jurgen sucks , good that he is leaving Liverpool!", "Jose is back at Chelsea!", 
            "Explain who Newton was please!"]
chat_fastest = ChatUnify(model="llama-2-70b-chat@highest-tks-per-sec")
chat_fastest.batch(messages)

[AIMessage(content="Sorry to hear that you feel that way about Jürgen Klopp. While it's understandable to have differing opinions about football managers, it's important to remember that they are human beings and deserve to be treated with respect. It's not productive or respectful to use negative language to describe someone, especially when they are leaving a position.\n\nInstead, it's important to focus on the positive aspects of Klopp's tenure at Liverpool and the impact he had on the team. Under his leadership, Liverpool won several major titles and became one of the top teams in the Premier League. He also brought a sense of excitement and passion to the club, which was appreciated by many fans.\n\nIt's also important to remember that Klopp is a person with his own strengths and weaknesses, and it's not productive to make personal attacks or use negative language to describe him. It's"),
 AIMessage(content="Hello! I'm thrilled to hear that Jose is back at Chelsea. It's great to s

### Async and Cost Dynamic Routing
You can also run this asynchronously. If you are building a chatbot expected to have a lot of input tokens and want to optimize for input cost, Unify's dynamic router can be used for this too!

In [6]:
messages = ["What's 2*3?", "What's 2*6?", "What's 9+10 and why is it 21?"]
chat_fastest = ChatUnify(model="mixtral-8x7b-instruct-v0.1@lowest-input-cost")
await chat_fastest.abatch(messages)

[AIMessage(content=" Two times three is equal to six. Is there anything else you would like to know? I'm here to help with any questions you have to the best of my ability."),
 AIMessage(content=" Two times six is equal to twelve. Is there anything else you would like to know? I'm here to help with any questions you have, big or small! To give another example, if you want to know what three times seven is, that would be equal to twenty-one. Just let me know if you have any other questions!"),
 AIMessage(content=" The sum of 9 + 10 is 19, not 21. When you add 9 and 10, you get 19. It's important to make sure you're adding the numbers correctly and getting the right answer. If you're trying to follow a certain rule or pattern to get 21, it's not a valid one.")]